In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import sympy as sym

import models
import payoffs
import plotting
import selection_functions
import symbolics

In [2]:
%matplotlib inline
plt.style.use("ggplot")

# Monomorphic $\gamma$ models

## Only G allele present 

Set $x_2=1-x_1$, $x_3=x4=0$ so that only the G allele of the $\gamma$ gene is present

In [3]:
x1, x2, x3 = sym.symbols("x1, x2, x3", real=True, nonnegative=True)
x4 = 1 - x1 - x2 - x3
T, R, P, S = sym.symbols('T, R, P, S', real=True, positive=True)
M, m = sym.symbols("M, m", real=True, nonnegative=True)
epsilon = sym.symbols("epsilon", real=True, positive=True)

UGA = symbolics.UGA
UgA = symbolics.UgA

In [4]:
x = np.array([[x1], [1 - x1], [0], [0]])
payoff_kernel = np.array([[R, S], [T, P]])
W = models.generalized_sexual_selection(x, UGA, UgA, payoff_kernel, M, m, epsilon)

In [5]:
(f1,), *_ = models.offspring_genotypes_evolution(W, x)

## The locus of potential equilibria (LPE)

In [6]:
UGA_star = sym.symbols("UGA_star", real=True, nonnegative=True)

In [7]:
# solving for the LPE using Python
x1_star, = sym.solve(f1, x1, implicit=True)
x1_star =  sym.cancel(x1_star.subs({UGA(x1): UGA_star}))
x1_star

-1.0*(4.0*M*UGA_star*epsilon - 2.0*M*UGA_star - 4.0*M*epsilon + 4.0*P*UGA_star**2*epsilon - 8.0*P*UGA_star*epsilon + 4.0*P*epsilon + 2.0*R*UGA_star**2 - 2.0*S*UGA_star**2 + 2.0*S*UGA_star - 4.0*T*UGA_star**2*epsilon + 4.0*T*UGA_star*epsilon)/(4.0*M*epsilon + 2.0*M - 4.0*P*UGA_star**2*epsilon - 2.0*P*UGA_star**2 + 8.0*P*UGA_star*epsilon + 4.0*P*UGA_star - 4.0*P*epsilon - 2.0*P - 4.0*R*UGA_star**2*epsilon - 2.0*R*UGA_star**2 + 4.0*S*UGA_star**2*epsilon + 2.0*S*UGA_star**2 - 4.0*S*UGA_star*epsilon - 2.0*S*UGA_star + 4.0*T*UGA_star**2*epsilon + 2.0*T*UGA_star**2 - 4.0*T*UGA_star*epsilon - 2.0*T*UGA_star)

## Invadability conditions

In [8]:
x = np.array([[x1], [x2], [x3], [x4]])
W = models.generalized_sexual_selection(x, UGA, UgA, payoff_kernel, M, m, epsilon)
(f1,), (f2,), (f3,), _ = models.offspring_genotypes_evolution(W, x)

In [9]:
F = sym.Matrix([f1, f2, f3])

In [10]:
UGA_star = sym.symbols("UGA_star", nonnegative=True, real=True)
UGA_prime_star = sym.symbols("UGA_prime_star", real=True, nonnegative=True)
UgA_star = sym.symbols("UgA_star", nonnegative=True, real=True)

F_jac = F.jacobian([x1, x2, x3])
monomorphic_model_jac = (F_jac.subs({x2: 1 - x1, x3:0})
                              .doit()
                              .subs({UGA(x1): UGA_star, sym.Derivative(UGA(x1)): UGA_prime_star, UgA(x1): UgA_star}))



In [19]:
simplified_monomorphic_model_jac = sym.zeros(3, 3)

for i in range(3):
    for j in range(3):
        simplified_monomorphic_model_jac[i, j] = sym.factor(sym.cancel(sym.together(monomorphic_model_jac[i,j])))
        print("Finished with element {},{}!".format(i,j))
        

Finished with element 0,0!
Finished with element 0,1!
Finished with element 0,2!
Finished with element 1,0!
Finished with element 1,1!
Finished with element 1,2!
Finished with element 2,0!
Finished with element 2,1!
Finished with element 2,2!


In [22]:
# this takes ages to finish!
eigenvals = simplified_monomorphic_model_jac.eigenvals()

In [23]:
# so I have serialized the results to disk...
import pickle

with open('non-invadable-eigenvalues.pickle', 'wb') as handle:
    pickle.dump(eigenvals, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [125]:
# ...so that they can be loaded on demand!
with open('non-invadable-eigenvalues.pickle', 'rb') as pickled_eigenvals:
    eigenvals = pickle.load(pickled_eigenvals)

In [126]:
e1, e2, e3 = eigenvals.keys()

In [145]:
sym.limit(e2.subs({x1: x1_star}).subs({epsilon: 0}), UGA_star, 1)

KeyboardInterrupt: 

In [149]:
sym.factor(sym.together(e2.subs({x1: x1_star}).subs({epsilon: 0})))

KeyboardInterrupt: 

In [153]:
sym.cancel(sym.together(e2.subs({x1: x1_star}).subs({epsilon: 0})))

KeyboardInterrupt: 

In [136]:
# derive limiting behavior of eigenvalues as 
limit_e2_as_UGA_star_zero = sym.limit(e2.subs({x1: x1_star}).subs({epsilon: 0}), UGA_star,  0)
limit_e3_as_UGA_star_zero = sym.limit(e3.subs({x1: x1_star}).subs({epsilon: 0}), UGA_star,  0)

1.0*(0.125*P**(5/2)*S + 0.375*P*P**(3/2)*S*UgA_star**2 - 0.75*P*P**(3/2)*S*UgA_star - 0.375*P*P**(3/2)*S + 0.375*P**(3/2)*R*S*UgA_star**2 - 0.375*P**(3/2)*S*S*UgA_star**2 + 0.375*P**(3/2)*S*S*UgA_star - 0.375*P**(3/2)*S*T*UgA_star**2 + 0.375*P**(3/2)*S*T*UgA_star + 0.125*P*sqrt(1.0*P*P**2*S**2*UgA_star**4 - 4.0*P*P**2*S**2*UgA_star**3 + 10.0*P*P**2*S**2*UgA_star**2 - 12.0*P*P**2*S**2*UgA_star + 9.0*P*P**2*S**2 - 2.0*P*P**2*S*S*UgA_star**2 + 4.0*P*P**2*S*S*UgA_star - 6.0*P*P**2*S*S + 2.0*P*P*R*S**2*UgA_star**4 - 4.0*P*P*R*S**2*UgA_star**3 - 2.0*P*P*R*S**2*UgA_star**2 - 2.0*P*P*S*S**2*UgA_star**4 + 6.0*P*P*S*S**2*UgA_star**3 - 2.0*P*P*S*S**2*UgA_star**2 - 2.0*P*P*S*S**2*UgA_star - 2.0*P*P*S**2*T*UgA_star**4 + 6.0*P*P*S**2*T*UgA_star**3 - 10.0*P*P*S**2*T*UgA_star**2 + 6.0*P*P*S**2*T*UgA_star + P**3*S**2 + 6.0*P**2*R*S*S*UgA_star**2 - 6.0*P**2*S*S**2*UgA_star**2 + 6.0*P**2*S*S**2*UgA_star + 2.0*P**2*S*S*T*UgA_star**2 - 2.0*P**2*S*S*T*UgA_star + 1.0*P*R**2*S**2*UgA_star**4 - 2.0*P*R*S*S**2*

In [ ]:
limit_e2_as_UGA_star_one = sym.limit(e2.subs({x1: x1_star}).subs({epsilon: 0}), UGA_star,  1)
limit_e3_as_UGA_star_one = sym.limit(e3.subs({x1: x1_star}).subs({epsilon: 0}), UGA_star,  1)

In [137]:
delta = (1.0*P*P**2*S**2*UgA_star**4 - 4.0*P*P**2*S**2*UgA_star**3 + 10.0*P*P**2*S**2*UgA_star**2 - 12.0*P*P**2*S**2*UgA_star + 9.0*P*P**2*S**2 - 2.0*P*P**2*S*S*UgA_star**2 + 4.0*P*P**2*S*S*UgA_star - 6.0*P*P**2*S*S + 2.0*P*P*R*S**2*UgA_star**4 - 4.0*P*P*R*S**2*UgA_star**3 - 2.0*P*P*R*S**2*UgA_star**2 - 2.0*P*P*S*S**2*UgA_star**4 + 6.0*P*P*S*S**2*UgA_star**3 - 2.0*P*P*S*S**2*UgA_star**2 - 2.0*P*P*S*S**2*UgA_star - 2.0*P*P*S**2*T*UgA_star**4 + 6.0*P*P*S**2*T*UgA_star**3 - 10.0*P*P*S**2*T*UgA_star**2 + 6.0*P*P*S**2*T*UgA_star + P**3*S**2 + 6.0*P**2*R*S*S*UgA_star**2 - 6.0*P**2*S*S**2*UgA_star**2 + 6.0*P**2*S*S**2*UgA_star + 2.0*P**2*S*S*T*UgA_star**2 - 2.0*P**2*S*S*T*UgA_star + 1.0*P*R**2*S**2*UgA_star**4 - 2.0*P*R*S*S**2*UgA_star**4 + 2.0*P*R*S*S**2*UgA_star**3 - 2.0*P*R*S**2*T*UgA_star**4 + 2.0*P*R*S**2*T*UgA_star**3 + 1.0*P*S**2*S**2*UgA_star**4 - 2.0*P*S**2*S**2*UgA_star**3 + 1.0*P*S**2*S**2*UgA_star**2 + 2.0*P*S*S**2*T*UgA_star**4 - 4.0*P*S*S**2*T*UgA_star**3 + 2.0*P*S*S**2*T*UgA_star**2 + 1.0*P*S**2*T**2*UgA_star**4 - 2.0*P*S**2*T**2*UgA_star**3 + 1.0*P*S**2*T**2*UgA_star**2)

In [138]:
sym.factor(delta)

1.0*P*S**2*(1.0*P*UgA_star**2 - 2.0*P*UgA_star + 2.0*P + 1.0*R*UgA_star**2 - 1.0*S*UgA_star**2 + 1.0*S*UgA_star - 1.0*T*UgA_star**2 + 1.0*T*UgA_star)**2

In [ ]:
-(32.0*M*epsilon**2*(-M + P)**2 + 112.0*M*epsilon*(-M + P)**2 + 8.0*M*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 24.0*M*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 32.0*P*UgA_star**2*epsilon**2*(-M + P)**2 + 48.0*P*UgA_star**2*epsilon*(-M + P)**2 - 8.0*P*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 12.0*P*UgA_star**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*P*UgA_star*epsilon**2*(-M + P)**2 - 96.0*P*UgA_star*epsilon*(-M + P)**2 + 16.0*P*UgA_star*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 24.0*P*UgA_star*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 64.0*P*epsilon**2*(-M + P)**2 - 64.0*P*epsilon*(-M + P)**2 - 16.0*P*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 12.0*P*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 32.0*R*UgA_star**2*epsilon**2*(-M + P)**2 + 48.0*R*UgA_star**2*epsilon*(-M + P)**2 - 8.0*R*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 12.0*R*UgA_star**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 32.0*S*UgA_star**2*epsilon**2*(-M + P)**2 - 48.0*S*UgA_star**2*epsilon*(-M + P)**2 + 8.0*S*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 12.0*S*UgA_star**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 32.0*S*UgA_star*epsilon**2*(-M + P)**2 + 48.0*S*UgA_star*epsilon*(-M + P)**2 - 8.0*S*UgA_star*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 12.0*S*UgA_star*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 32.0*T*UgA_star**2*epsilon**2*(-M + P)**2 - 48.0*T*UgA_star**2*epsilon*(-M + P)**2 + 8.0*T*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 12.0*T*UgA_star**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 32.0*T*UgA_star*epsilon**2*(-M + P)**2 + 48.0*T*UgA_star*epsilon*(-M + P)**2 - 8.0*T*UgA_star*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 12.0*T*UgA_star*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 32.0*epsilon**2*m*(-M + P)**2 - 48.0*epsilon*m*(-M + P)**2 + 8.0*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 12.0*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - sqrt(1024.0*M**2*epsilon**4*(-M + P)**2 - 1024.0*M**2*epsilon**3*(-M + P)**2 + 512.0*M**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*M**2*epsilon**2*(-M + P)**2 - 768.0*M**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*M**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 256.0*M**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*M**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 64.0*M**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*M*P*UgA_star**2*epsilon**4*(-M + P)**2 - 1024.0*M*P*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*M*P*UgA_star**2*epsilon**2*(-M + P)**2 + 256.0*M*P*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*M*P*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 128.0*M*P*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*M*P*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 64.0*M*P*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 4096.0*M*P*UgA_star*epsilon**4*(-M + P)**2 + 2048.0*M*P*UgA_star*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 1024.0*M*P*UgA_star*epsilon**2*(-M + P)**2 - 512.0*M*P*UgA_star*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*M*P*UgA_star*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 256.0*M*P*UgA_star*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*M*P*UgA_star*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 128.0*M*P*UgA_star*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 4096.0*M*P*epsilon**4*(-M + P)**2 + 2048.0*M*P*epsilon**3*(-M + P)**2 - 2048.0*M*P*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 1792.0*M*P*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 256.0*M*P*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 640.0*M*P*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 320.0*M*P*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 192.0*M*P*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*M*R*UgA_star**2*epsilon**4*(-M + P)**2 - 1024.0*M*R*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*M*R*UgA_star**2*epsilon**2*(-M + P)**2 - 768.0*M*R*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*M*R*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 384.0*M*R*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 192.0*M*R*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 64.0*M*R*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 2048.0*M*S*UgA_star**2*epsilon**4*(-M + P)**2 + 1024.0*M*S*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*M*S*UgA_star**2*epsilon**2*(-M + P)**2 + 768.0*M*S*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*M*S*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 384.0*M*S*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 192.0*M*S*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 64.0*M*S*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*M*S*UgA_star*epsilon**4*(-M + P)**2 - 1024.0*M*S*UgA_star*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*M*S*UgA_star*epsilon**2*(-M + P)**2 - 768.0*M*S*UgA_star*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*M*S*UgA_star*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 384.0*M*S*UgA_star*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 192.0*M*S*UgA_star*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 64.0*M*S*UgA_star*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 2048.0*M*T*UgA_star**2*epsilon**4*(-M + P)**2 + 1024.0*M*T*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*M*T*UgA_star**2*epsilon**2*(-M + P)**2 - 256.0*M*T*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*M*T*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 128.0*M*T*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 64.0*M*T*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 64.0*M*T*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*M*T*UgA_star*epsilon**4*(-M + P)**2 - 1024.0*M*T*UgA_star*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*M*T*UgA_star*epsilon**2*(-M + P)**2 + 256.0*M*T*UgA_star*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*M*T*UgA_star*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 128.0*M*T*UgA_star*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*M*T*UgA_star*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 64.0*M*T*UgA_star*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 1024.0*M*UgA_star*epsilon**2*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*M*UgA_star*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*M*UgA_star*epsilon*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 128.0*M*UgA_star*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 2048.0*M*epsilon**4*m*(-M + P)**2 + 1024.0*M*epsilon**3*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*M*epsilon**2*m*(-M + P)**2 - 256.0*M*epsilon**2*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*M*epsilon**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 128.0*M*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 64.0*M*epsilon*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 64.0*M*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 1024.0*P**2*UgA_star**4*epsilon**4*(-M + P)**2 + 1024.0*P**2*UgA_star**4*epsilon**3*(-M + P)**2 + 512.0*P**2*UgA_star**4*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*P**2*UgA_star**4*epsilon**2*(-M + P)**2 + 512.0*P**2*UgA_star**4*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*P**2*UgA_star**4*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 128.0*P**2*UgA_star**4*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*P**2*UgA_star**4*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 16.0*P**2*UgA_star**4*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 4096.0*P**2*UgA_star**3*epsilon**4*(-M + P)**2 - 4096.0*P**2*UgA_star**3*epsilon**3*(-M + P)**2 - 2048.0*P**2*UgA_star**3*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 1024.0*P**2*UgA_star**3*epsilon**2*(-M + P)**2 - 2048.0*P**2*UgA_star**3*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 256.0*P**2*UgA_star**3*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 512.0*P**2*UgA_star**3*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 256.0*P**2*UgA_star**3*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 64.0*P**2*UgA_star**3*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 8192.0*P**2*UgA_star**2*epsilon**4*(-M + P)**2 + 6144.0*P**2*UgA_star**2*epsilon**3*(-M + P)**2 + 4096.0*P**2*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 1024.0*P**2*UgA_star**2*epsilon**2*(-M + P)**2 + 2816.0*P**2*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*P**2*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 896.0*P**2*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 320.0*P**2*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 160.0*P**2*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 8192.0*P**2*UgA_star*epsilon**4*(-M + P)**2 - 4096.0*P**2*UgA_star*epsilon**3*(-M + P)**2 - 4096.0*P**2*UgA_star*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 1536.0*P**2*UgA_star*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*P**2*UgA_star*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 768.0*P**2*UgA_star*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*P**2*UgA_star*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 192.0*P**2*UgA_star*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 4096.0*P**2*epsilon**4*(-M + P)**2 + 2048.0*P**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*P**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*P**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 512.0*P**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*P**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 144.0*P**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 2048.0*P*R*UgA_star**4*epsilon**4*(-M + P)**2 + 2048.0*P*R*UgA_star**4*epsilon**3*(-M + P)**2 + 1024.0*P*R*UgA_star**4*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*P*R*UgA_star**4*epsilon**2*(-M + P)**2 + 1024.0*P*R*UgA_star**4*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*P*R*UgA_star**4*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 256.0*P*R*UgA_star**4*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*P*R*UgA_star**4*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 32.0*P*R*UgA_star**4*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 4096.0*P*R*UgA_star**3*epsilon**4*(-M + P)**2 - 4096.0*P*R*UgA_star**3*epsilon**3*(-M + P)**2 - 2048.0*P*R*UgA_star**3*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 1024.0*P*R*UgA_star**3*epsilon**2*(-M + P)**2 - 2048.0*P*R*UgA_star**3*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 256.0*P*R*UgA_star**3*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 512.0*P*R*UgA_star**3*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 256.0*P*R*UgA_star**3*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 64.0*P*R*UgA_star**3*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 4096.0*P*R*UgA_star**2*epsilon**4*(-M + P)**2 + 2048.0*P*R*UgA_star**2*epsilon**3*(-M + P)**2 + 2048.0*P*R*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 1792.0*P*R*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*P*R*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 128.0*P*R*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 320.0*P*R*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*P*R*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*P*S*UgA_star**4*epsilon**4*(-M + P)**2 - 2048.0*P*S*UgA_star**4*epsilon**3*(-M + P)**2 - 1024.0*P*S*UgA_star**4*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*P*S*UgA_star**4*epsilon**2*(-M + P)**2 - 1024.0*P*S*UgA_star**4*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*P*S*UgA_star**4*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 256.0*P*S*UgA_star**4*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*P*S*UgA_star**4*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*P*S*UgA_star**4*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 6144.0*P*S*UgA_star**3*epsilon**4*(-M + P)**2 + 6144.0*P*S*UgA_star**3*epsilon**3*(-M + P)**2 + 3072.0*P*S*UgA_star**3*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 1536.0*P*S*UgA_star**3*epsilon**2*(-M + P)**2 + 3072.0*P*S*UgA_star**3*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 384.0*P*S*UgA_star**3*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 768.0*P*S*UgA_star**3*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 384.0*P*S*UgA_star**3*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 96.0*P*S*UgA_star**3*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 8192.0*P*S*UgA_star**2*epsilon**4*(-M + P)**2 - 6144.0*P*S*UgA_star**2*epsilon**3*(-M + P)**2 - 4096.0*P*S*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 1024.0*P*S*UgA_star**2*epsilon**2*(-M + P)**2 - 3840.0*P*S*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*P*S*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 384.0*P*S*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 576.0*P*S*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*P*S*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 4096.0*P*S*UgA_star*epsilon**4*(-M + P)**2 + 2048.0*P*S*UgA_star*epsilon**3*(-M + P)**2 + 2048.0*P*S*UgA_star*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 1792.0*P*S*UgA_star*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*P*S*UgA_star*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 128.0*P*S*UgA_star*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 320.0*P*S*UgA_star*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*P*S*UgA_star*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*P*T*UgA_star**4*epsilon**4*(-M + P)**2 - 2048.0*P*T*UgA_star**4*epsilon**3*(-M + P)**2 - 1024.0*P*T*UgA_star**4*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*P*T*UgA_star**4*epsilon**2*(-M + P)**2 - 1024.0*P*T*UgA_star**4*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*P*T*UgA_star**4*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 256.0*P*T*UgA_star**4*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*P*T*UgA_star**4*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*P*T*UgA_star**4*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 6144.0*P*T*UgA_star**3*epsilon**4*(-M + P)**2 + 6144.0*P*T*UgA_star**3*epsilon**3*(-M + P)**2 + 3072.0*P*T*UgA_star**3*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 1536.0*P*T*UgA_star**3*epsilon**2*(-M + P)**2 + 3072.0*P*T*UgA_star**3*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 384.0*P*T*UgA_star**3*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 768.0*P*T*UgA_star**3*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 384.0*P*T*UgA_star**3*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 96.0*P*T*UgA_star**3*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 8192.0*P*T*UgA_star**2*epsilon**4*(-M + P)**2 - 6144.0*P*T*UgA_star**2*epsilon**3*(-M + P)**2 - 4096.0*P*T*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 1024.0*P*T*UgA_star**2*epsilon**2*(-M + P)**2 - 2816.0*P*T*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*P*T*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 896.0*P*T*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 320.0*P*T*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 160.0*P*T*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 4096.0*P*T*UgA_star*epsilon**4*(-M + P)**2 + 2048.0*P*T*UgA_star*epsilon**3*(-M + P)**2 + 2048.0*P*T*UgA_star*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 768.0*P*T*UgA_star*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*P*T*UgA_star*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 384.0*P*T*UgA_star*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*P*T*UgA_star*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 96.0*P*T*UgA_star*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*P*UgA_star**2*epsilon**4*m*(-M + P)**2 - 2048.0*P*UgA_star**2*epsilon**3*m*(-M + P)**2 - 1024.0*P*UgA_star**2*epsilon**3*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*P*UgA_star**2*epsilon**2*m*(-M + P)**2 - 1024.0*P*UgA_star**2*epsilon**2*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*P*UgA_star**2*epsilon**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 256.0*P*UgA_star**2*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*P*UgA_star**2*epsilon*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*P*UgA_star**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 4096.0*P*UgA_star*epsilon**4*m*(-M + P)**2 + 4096.0*P*UgA_star*epsilon**3*m*(-M + P)**2 + 2048.0*P*UgA_star*epsilon**3*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 1024.0*P*UgA_star*epsilon**2*m*(-M + P)**2 + 1024.0*P*UgA_star*epsilon**2*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*P*UgA_star*epsilon**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 1024.0*P*UgA_star*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 192.0*P*UgA_star*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 4096.0*P*epsilon**4*m*(-M + P)**2 - 2048.0*P*epsilon**3*m*(-M + P)**2 - 2048.0*P*epsilon**3*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 768.0*P*epsilon**2*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 256.0*P*epsilon**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 384.0*P*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 64.0*P*epsilon*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 96.0*P*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 1024.0*R**2*UgA_star**4*epsilon**4*(-M + P)**2 + 1024.0*R**2*UgA_star**4*epsilon**3*(-M + P)**2 + 512.0*R**2*UgA_star**4*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*R**2*UgA_star**4*epsilon**2*(-M + P)**2 + 512.0*R**2*UgA_star**4*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*R**2*UgA_star**4*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 128.0*R**2*UgA_star**4*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*R**2*UgA_star**4*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 16.0*R**2*UgA_star**4*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*R*S*UgA_star**4*epsilon**4*(-M + P)**2 - 2048.0*R*S*UgA_star**4*epsilon**3*(-M + P)**2 - 1024.0*R*S*UgA_star**4*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*R*S*UgA_star**4*epsilon**2*(-M + P)**2 - 1024.0*R*S*UgA_star**4*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*R*S*UgA_star**4*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 256.0*R*S*UgA_star**4*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*R*S*UgA_star**4*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*R*S*UgA_star**4*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 2048.0*R*S*UgA_star**3*epsilon**4*(-M + P)**2 + 2048.0*R*S*UgA_star**3*epsilon**3*(-M + P)**2 + 1024.0*R*S*UgA_star**3*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*R*S*UgA_star**3*epsilon**2*(-M + P)**2 + 1024.0*R*S*UgA_star**3*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*R*S*UgA_star**3*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 256.0*R*S*UgA_star**3*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*R*S*UgA_star**3*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 32.0*R*S*UgA_star**3*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*R*T*UgA_star**4*epsilon**4*(-M + P)**2 - 2048.0*R*T*UgA_star**4*epsilon**3*(-M + P)**2 - 1024.0*R*T*UgA_star**4*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*R*T*UgA_star**4*epsilon**2*(-M + P)**2 - 1024.0*R*T*UgA_star**4*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*R*T*UgA_star**4*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 256.0*R*T*UgA_star**4*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*R*T*UgA_star**4*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*R*T*UgA_star**4*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 2048.0*R*T*UgA_star**3*epsilon**4*(-M + P)**2 + 2048.0*R*T*UgA_star**3*epsilon**3*(-M + P)**2 + 1024.0*R*T*UgA_star**3*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*R*T*UgA_star**3*epsilon**2*(-M + P)**2 + 1024.0*R*T*UgA_star**3*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*R*T*UgA_star**3*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 256.0*R*T*UgA_star**3*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*R*T*UgA_star**3*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 32.0*R*T*UgA_star**3*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*R*UgA_star**2*epsilon**4*m*(-M + P)**2 - 2048.0*R*UgA_star**2*epsilon**3*m*(-M + P)**2 - 1024.0*R*UgA_star**2*epsilon**3*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*R*UgA_star**2*epsilon**2*m*(-M + P)**2 - 1024.0*R*UgA_star**2*epsilon**2*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*R*UgA_star**2*epsilon**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 256.0*R*UgA_star**2*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*R*UgA_star**2*epsilon*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*R*UgA_star**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 1024.0*S**2*UgA_star**4*epsilon**4*(-M + P)**2 + 1024.0*S**2*UgA_star**4*epsilon**3*(-M + P)**2 + 512.0*S**2*UgA_star**4*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*S**2*UgA_star**4*epsilon**2*(-M + P)**2 + 512.0*S**2*UgA_star**4*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*S**2*UgA_star**4*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 128.0*S**2*UgA_star**4*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*S**2*UgA_star**4*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 16.0*S**2*UgA_star**4*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*S**2*UgA_star**3*epsilon**4*(-M + P)**2 - 2048.0*S**2*UgA_star**3*epsilon**3*(-M + P)**2 - 1024.0*S**2*UgA_star**3*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*S**2*UgA_star**3*epsilon**2*(-M + P)**2 - 1024.0*S**2*UgA_star**3*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*S**2*UgA_star**3*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 256.0*S**2*UgA_star**3*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*S**2*UgA_star**3*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*S**2*UgA_star**3*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 1024.0*S**2*UgA_star**2*epsilon**4*(-M + P)**2 + 1024.0*S**2*UgA_star**2*epsilon**3*(-M + P)**2 + 512.0*S**2*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*S**2*UgA_star**2*epsilon**2*(-M + P)**2 + 512.0*S**2*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*S**2*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 128.0*S**2*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*S**2*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 16.0*S**2*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 2048.0*S*T*UgA_star**4*epsilon**4*(-M + P)**2 + 2048.0*S*T*UgA_star**4*epsilon**3*(-M + P)**2 + 1024.0*S*T*UgA_star**4*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*S*T*UgA_star**4*epsilon**2*(-M + P)**2 + 1024.0*S*T*UgA_star**4*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*S*T*UgA_star**4*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 256.0*S*T*UgA_star**4*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*S*T*UgA_star**4*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 32.0*S*T*UgA_star**4*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 4096.0*S*T*UgA_star**3*epsilon**4*(-M + P)**2 - 4096.0*S*T*UgA_star**3*epsilon**3*(-M + P)**2 - 2048.0*S*T*UgA_star**3*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 1024.0*S*T*UgA_star**3*epsilon**2*(-M + P)**2 - 2048.0*S*T*UgA_star**3*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 256.0*S*T*UgA_star**3*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 512.0*S*T*UgA_star**3*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 256.0*S*T*UgA_star**3*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 64.0*S*T*UgA_star**3*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 2048.0*S*T*UgA_star**2*epsilon**4*(-M + P)**2 + 2048.0*S*T*UgA_star**2*epsilon**3*(-M + P)**2 + 1024.0*S*T*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*S*T*UgA_star**2*epsilon**2*(-M + P)**2 + 1024.0*S*T*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*S*T*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 256.0*S*T*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*S*T*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 32.0*S*T*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 2048.0*S*UgA_star**2*epsilon**4*m*(-M + P)**2 + 2048.0*S*UgA_star**2*epsilon**3*m*(-M + P)**2 + 1024.0*S*UgA_star**2*epsilon**3*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*S*UgA_star**2*epsilon**2*m*(-M + P)**2 + 1024.0*S*UgA_star**2*epsilon**2*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*S*UgA_star**2*epsilon**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 256.0*S*UgA_star**2*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*S*UgA_star**2*epsilon*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 32.0*S*UgA_star**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*S*UgA_star*epsilon**4*m*(-M + P)**2 - 2048.0*S*UgA_star*epsilon**3*m*(-M + P)**2 - 1024.0*S*UgA_star*epsilon**3*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*S*UgA_star*epsilon**2*m*(-M + P)**2 - 1024.0*S*UgA_star*epsilon**2*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*S*UgA_star*epsilon**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 256.0*S*UgA_star*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*S*UgA_star*epsilon*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*S*UgA_star*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 1024.0*T**2*UgA_star**4*epsilon**4*(-M + P)**2 + 1024.0*T**2*UgA_star**4*epsilon**3*(-M + P)**2 + 512.0*T**2*UgA_star**4*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*T**2*UgA_star**4*epsilon**2*(-M + P)**2 + 512.0*T**2*UgA_star**4*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*T**2*UgA_star**4*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 128.0*T**2*UgA_star**4*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*T**2*UgA_star**4*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 16.0*T**2*UgA_star**4*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*T**2*UgA_star**3*epsilon**4*(-M + P)**2 - 2048.0*T**2*UgA_star**3*epsilon**3*(-M + P)**2 - 1024.0*T**2*UgA_star**3*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*T**2*UgA_star**3*epsilon**2*(-M + P)**2 - 1024.0*T**2*UgA_star**3*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*T**2*UgA_star**3*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 256.0*T**2*UgA_star**3*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*T**2*UgA_star**3*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*T**2*UgA_star**3*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 1024.0*T**2*UgA_star**2*epsilon**4*(-M + P)**2 + 1024.0*T**2*UgA_star**2*epsilon**3*(-M + P)**2 + 512.0*T**2*UgA_star**2*epsilon**3*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*T**2*UgA_star**2*epsilon**2*(-M + P)**2 + 512.0*T**2*UgA_star**2*epsilon**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*T**2*UgA_star**2*epsilon**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 128.0*T**2*UgA_star**2*epsilon*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*T**2*UgA_star**2*epsilon*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 16.0*T**2*UgA_star**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 2048.0*T*UgA_star**2*epsilon**4*m*(-M + P)**2 + 2048.0*T*UgA_star**2*epsilon**3*m*(-M + P)**2 + 1024.0*T*UgA_star**2*epsilon**3*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 512.0*T*UgA_star**2*epsilon**2*m*(-M + P)**2 + 1024.0*T*UgA_star**2*epsilon**2*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*T*UgA_star**2*epsilon**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 256.0*T*UgA_star**2*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 128.0*T*UgA_star**2*epsilon*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 32.0*T*UgA_star**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 2048.0*T*UgA_star*epsilon**4*m*(-M + P)**2 - 2048.0*T*UgA_star*epsilon**3*m*(-M + P)**2 - 1024.0*T*UgA_star*epsilon**3*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 512.0*T*UgA_star*epsilon**2*m*(-M + P)**2 - 1024.0*T*UgA_star*epsilon**2*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*T*UgA_star*epsilon**2*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 256.0*T*UgA_star*epsilon*m*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*T*UgA_star*epsilon*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 - 32.0*T*UgA_star*m*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 1024.0*epsilon**4*m**2*(-M + P)**2 + 1024.0*epsilon**3*m**2*(-M + P)**2 + 512.0*epsilon**3*m**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 256.0*epsilon**2*m**2*(-M + P)**2 + 512.0*epsilon**2*m**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*epsilon**2*m**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 128.0*epsilon*m**2*(-M + P)*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) + 64.0*epsilon*m**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2 + 16.0*m**2*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)**2)*Abs(M - P))/((-M + P)*(128.0*M*epsilon*(-M + P) + 32.0*M*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*P*epsilon*(-M + P) - 32.0*P*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P)))

In [131]:
((-M + P)*(128.0*M*epsilon*(-M + P) + 32.0*M*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P) - 128.0*P*epsilon*(-M + P) - 32.0*P*(4.0*M*epsilon + 2.0*M - 4.0*P*epsilon - 2.0*P))).subs({epsilon: 0})

(-M + P)*(32.0*M*(2.0*M - 2.0*P) - 32.0*P*(2.0*M - 2.0*P))

In [154]:
sym.limit(e2.subs({x1: x1_star}).subs({T: 10, R: 5, P: 4, S: 1, M: 0, m: 0, epsilon: 0}), UGA_star, 1)

-inf*I

In [76]:
delta = 256.0*P**2*UgA_star**4*epsilon**2 + 256.0*P**2*UgA_star**4*epsilon + 64.0*P**2*UgA_star**4 - 1024.0*P**2*UgA_star**3*epsilon**2 - 1024.0*P**2*UgA_star**3*epsilon - 256.0*P**2*UgA_star**3 + 1536.0*P**2*UgA_star**2*epsilon**2 + 1536.0*P**2*UgA_star**2*epsilon + 384.0*P**2*UgA_star**2 - 1024.0*P**2*UgA_star*epsilon**2 - 1024.0*P**2*UgA_star*epsilon - 256.0*P**2*UgA_star + 256.0*P**2*epsilon**2 + 256.0*P**2*epsilon + 64.0*P**2 + 512.0*P*R*UgA_star**4*epsilon**2 + 512.0*P*R*UgA_star**4*epsilon + 128.0*P*R*UgA_star**4 - 1024.0*P*R*UgA_star**3*epsilon**2 - 1024.0*P*R*UgA_star**3*epsilon - 256.0*P*R*UgA_star**3 + 2560.0*P*R*UgA_star**2*epsilon**2 + 1024.0*P*R*UgA_star**2*epsilon - 128.0*P*R*UgA_star**2 - 4096.0*P*R*UgA_star*epsilon**2 - 1024.0*P*R*UgA_star*epsilon + 512.0*P*R*UgA_star + 2048.0*P*R*epsilon**2 + 512.0*P*R*epsilon - 256.0*P*R - 512.0*P*S*UgA_star**4*epsilon**2 - 512.0*P*S*UgA_star**4*epsilon - 128.0*P*S*UgA_star**4 + 1536.0*P*S*UgA_star**3*epsilon**2 + 1536.0*P*S*UgA_star**3*epsilon + 384.0*P*S*UgA_star**3 - 1536.0*P*S*UgA_star**2*epsilon**2 - 1536.0*P*S*UgA_star**2*epsilon - 384.0*P*S*UgA_star**2 + 512.0*P*S*UgA_star*epsilon**2 + 512.0*P*S*UgA_star*epsilon + 128.0*P*S*UgA_star - 512.0*P*T*UgA_star**4*epsilon**2 - 512.0*P*T*UgA_star**4*epsilon - 128.0*P*T*UgA_star**4 + 1536.0*P*T*UgA_star**3*epsilon**2 + 1536.0*P*T*UgA_star**3*epsilon + 384.0*P*T*UgA_star**3 - 1536.0*P*T*UgA_star**2*epsilon**2 - 1536.0*P*T*UgA_star**2*epsilon - 384.0*P*T*UgA_star**2 + 512.0*P*T*UgA_star*epsilon**2 + 512.0*P*T*UgA_star*epsilon + 128.0*P*T*UgA_star + 256.0*R**2*UgA_star**4*epsilon**2 + 256.0*R**2*UgA_star**4*epsilon + 64.0*R**2*UgA_star**4 + 512.0*R**2*UgA_star**2*epsilon + 256.0*R**2*UgA_star**2 + 256.0*R**2 - 512.0*R*S*UgA_star**4*epsilon**2 - 512.0*R*S*UgA_star**4*epsilon - 128.0*R*S*UgA_star**4 + 512.0*R*S*UgA_star**3*epsilon**2 + 512.0*R*S*UgA_star**3*epsilon + 128.0*R*S*UgA_star**3 - 512.0*R*S*UgA_star**2*epsilon - 256.0*R*S*UgA_star**2 + 512.0*R*S*UgA_star*epsilon + 256.0*R*S*UgA_star - 512.0*R*T*UgA_star**4*epsilon**2 - 512.0*R*T*UgA_star**4*epsilon - 128.0*R*T*UgA_star**4 + 512.0*R*T*UgA_star**3*epsilon**2 + 512.0*R*T*UgA_star**3*epsilon + 128.0*R*T*UgA_star**3 - 2048.0*R*T*UgA_star**2*epsilon**2 - 512.0*R*T*UgA_star**2*epsilon + 256.0*R*T*UgA_star**2 + 2048.0*R*T*UgA_star*epsilon**2 + 512.0*R*T*UgA_star*epsilon - 256.0*R*T*UgA_star + 256.0*S**2*UgA_star**4*epsilon**2 + 256.0*S**2*UgA_star**4*epsilon + 64.0*S**2*UgA_star**4 - 512.0*S**2*UgA_star**3*epsilon**2 - 512.0*S**2*UgA_star**3*epsilon - 128.0*S**2*UgA_star**3 + 256.0*S**2*UgA_star**2*epsilon**2 + 256.0*S**2*UgA_star**2*epsilon + 64.0*S**2*UgA_star**2 + 512.0*S*T*UgA_star**4*epsilon**2 + 512.0*S*T*UgA_star**4*epsilon + 128.0*S*T*UgA_star**4 - 1024.0*S*T*UgA_star**3*epsilon**2 - 1024.0*S*T*UgA_star**3*epsilon - 256.0*S*T*UgA_star**3 + 512.0*S*T*UgA_star**2*epsilon**2 + 512.0*S*T*UgA_star**2*epsilon + 128.0*S*T*UgA_star**2 + 256.0*T**2*UgA_star**4*epsilon**2 + 256.0*T**2*UgA_star**4*epsilon + 64.0*T**2*UgA_star**4 - 512.0*T**2*UgA_star**3*epsilon**2 - 512.0*T**2*UgA_star**3*epsilon - 128.0*T**2*UgA_star**3 + 256.0*T**2*UgA_star**2*epsilon**2 + 256.0*T**2*UgA_star**2*epsilon + 64.0*T**2*UgA_star**2

In [84]:
sym.factor(delta.subs({epsilon: 0, T: 25, R: 3, P: 2, S: 1, UgA_star: 0}))

1024.00000000000

In [34]:
# vectorized numeric repr for the eigenvalue
_numeric_e1 = sym.lambdify((UGA_star, UGA_prime_star, T, R, P, S, M, m, epsilon),
                           e1.subs({x1: x1_star}),
                           modules="numpy")

_numeric_e2 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, M, m, epsilon),
                           e2.subs({x1: x1_star}),
                           modules="numpy")

_numeric_e3 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, M, m, epsilon),
                           e3.subs({x1: x1_star}),
                           modules="numpy")

In [94]:
def _plot_first_eigenvalue(ax, vmin, vmax, T, R, P, S, M, m, epsilon):
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{GA}^{'*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_1$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100).reshape(-1, 1)
    equilibrium_selection_derivs = np.linspace(0, 5, 500).reshape(1, -1)
    Z = _numeric_e1(equilibrium_selection_probs, equilibrium_selection_derivs, T, R, P, S, M, m, epsilon)
    cax = ax.imshow(Z, origin="lower", aspect="auto", vmin=vmin, vmax=vmax)
    levels = np.arange(-0.25, vmax, 0.25)
    contours = ax.contour(Z, levels=levels, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs = ax.get_xticks()
    ax.set_xticks(locs[1:])
    ax.set_xticklabels(np.linspace(0, 5, locs.size-1))
    locs = ax.get_yticks()
    ax.set_yticks(locs[1:])
    ax.set_yticklabels(np.linspace(0, 1, locs.size-1))


def _plot_second_eigenvalue(ax, vmin, vmax, T, R, P, S, M, m, epsilon):

    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_2$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e2(UGAs, UgAs, T, R, P, S, M, m, epsilon)
    cax = ax.imshow(Z, origin="lower", aspect="auto", vmin=vmin, vmax=vmax)
    levels = np.arange(-0.25, vmax, 0.125)
    contours = ax.contour(Z, levels=levels, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs = ax.get_xticks()
    ax.set_xticks(locs[1:])
    ax.set_xticklabels(np.linspace(0, 1, locs.size-1))
    locs = ax.get_yticks()
    ax.set_yticks(locs[1:])
    ax.set_yticklabels(np.linspace(0, 1, locs.size-1))
    

def _plot_third_eigenvalue(ax, vmin, vmax, T, R, P, S, M, m, epsilon):

    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_3$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e3(UGAs, UgAs, T, R, P, S, M, m, epsilon)
    cax = ax.imshow(Z, origin="lower", aspect="auto", vmin=vmin, vmax=vmax)
    levels = np.arange(-0.5, vmax, 0.125)
    contours = ax.contour(Z, levels=levels, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs = ax.get_xticks()
    ax.set_xticks(locs[1:])
    ax.set_xticklabels(np.linspace(0, 1, locs.size-1))
    locs = ax.get_yticks()
    ax.set_yticks(locs[1:])
    ax.set_yticklabels(np.linspace(0, 1, locs.size-1))
    
    
def plot_eigenvalues(T, R, P, S, M, m, epsilon):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 9))
    vmin, vmax = -0.5, 1.5
    _plot_first_eigenvalue(ax1, vmin, vmax, T, R, P, S, M, m, epsilon)
    _plot_second_eigenvalue(ax2, vmin, vmax, T, R, P, S, M, m, epsilon)
    _plot_third_eigenvalue(ax3, vmin, vmax, T, R, P, S, M, m, epsilon)
    plt.show()
    

In [97]:
# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$M$")
m_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$m$")

e_slider = widgets.FloatSlider(value=0, min=0, max=1, step=1e-3, description=r"$\epsilon$", readout_format=".3f")

w = widgets.interactive(plot_eigenvalues, T=T_slider, R=R_slider, P=P_slider, S=S_slider,
                        M=M_slider, m=m_slider, epsilon=e_slider)
display(w)

A Jupyter Widget

In [168]:
togethered_e2 = sym.together(e2.subs({x1: x1_star}).subs({UGA_star: 1, m: 0}))

In [171]:
numerator_e2, denominator_e2 = sym.fraction(togethered_e2)

In [174]:
expanded_numerator_e2 = sym.expand(numerator_e2)

In [193]:
simplified_numerator_e2 = expanded_numerator_e2.subs({M**2: 0, epsilon**2: 0, M * epsilon: 0})

In [197]:
sym.factor(simplified_numerator_e2)

24.0*R**2*epsilon*(1.0*P*UgA_star**2 - 2.0*P*UgA_star + 1.0*P + 1.0*R*UgA_star**2 - 2.0*R - 1.0*S*UgA_star**2 + 1.0*S*UgA_star - 1.0*T*UgA_star**2 + 1.0*T*UgA_star)

In [195]:
sym.expand(denominator_e2)

-64.0*M**3*epsilon + 128.0*M**2*R*epsilon + 64.0*M**2*R*epsilon - 64.0*M*R**2*epsilon - 64.0*M*R*R*epsilon - 64.0*M*R*R*epsilon - 32.0*M*R*R + 32.0*M*R*R + 64.0*R*R**2*epsilon

In [198]:
simplified_denominator_e2 = sym.expand(denominator_e2).subs({M**2: 0, M * epsilon: 0})

In [200]:
simplified_denominator_e2

-32.0*M*R*R + 32.0*M*R*R + 64.0*R*R**2*epsilon

In [199]:
sym.factor(simplified_denominator_e2)

64.0*R*R**2*epsilon

In [202]:
sym.simplify(sym.factor(simplified_numerator_e2) / sym.factor(simplified_denominator_e2))

(0.375*P*UgA_star**2 - 0.75*P*UgA_star + 0.375*P + 0.375*R*UgA_star**2 - 0.75*R - 0.375*S*UgA_star**2 + 0.375*S*UgA_star - 0.375*T*UgA_star**2 + 0.375*T*UgA_star)/R

In [207]:
1000 / 5**3

8.0

In [209]:
3 / 8

0.375

\begin{align}
    \frac{3\big((R + P) - (T + S)\big)U_{gA}^{*2} + 3\big((T + S) - 2P\big)U_{gA}^* - 3\big(2R - P\big)}{8R} < 0 \\
    \big((R + P) - (T + S)\big)U_{gA}^{*2} + \big((T + S) - 2P\big)U_{gA}^* - \big(2R - P\big) < 0 \\
    \big((R + P) - (T + S)\big)\left(U_{gA}^* - \frac{1}{2}\frac{2P - (T + S)}{(R + P) - (T + S)}\right)^2 - (2R - P) - \frac{\big(2P - (T + S)\big)^2}{4\big((R + P) - (T + S)\big)} < 0\\
\end{align}

In [162]:
expanded_e2 = sym.expand(e2.subs({x1: x1_star}).subs({UGA_star: 1, m: 0}))

In [167]:
sym.together(expanded_e2)

(4.0*M*epsilon + 2.0*M - 4.0*R*epsilon - 2.0*R)*(16.0*M**2*epsilon**2 + 16.0*M**2*epsilon + 4.0*M**2 - 32.0*M*R*epsilon**2 - 32.0*M*R*epsilon - 8.0*M*R + 16.0*R**2*epsilon**2 + 16.0*R**2*epsilon + 4.0*R**2)*(-8.0*M**3*epsilon*(32.0*M**3*(4.0*M*epsilon + 2.0*M - 4.0*R*epsilon - 2.0*R) - 64.0*M**2*R*(4.0*M*epsilon + 2.0*M - 4.0*R*epsilon - 2.0*R) - 32.0*M**2*R*(4.0*M*epsilon + 2.0*M - 4.0*R*epsilon - 2.0*R) - 16.0*M**2*(16.0*M**2*epsilon**2 + 16.0*M**2*epsilon + 4.0*M**2 - 32.0*M*R*epsilon**2 - 32.0*M*R*epsilon - 8.0*M*R + 16.0*R**2*epsilon**2 + 16.0*R**2*epsilon + 4.0*R**2) + 32.0*M*R**2*(4.0*M*epsilon + 2.0*M - 4.0*R*epsilon - 2.0*R) + 64.0*M*R*R*(4.0*M*epsilon + 2.0*M - 4.0*R*epsilon - 2.0*R) + 16.0*M*R*(16.0*M**2*epsilon**2 + 16.0*M**2*epsilon + 4.0*M**2 - 32.0*M*R*epsilon**2 - 32.0*M*R*epsilon - 8.0*M*R + 16.0*R**2*epsilon**2 + 16.0*R**2*epsilon + 4.0*R**2) + 16.0*M*R*(16.0*M**2*epsilon**2 + 16.0*M**2*epsilon + 4.0*M**2 - 32.0*M*R*epsilon**2 - 32.0*M*R*epsilon - 8.0*M*R + 16.0*R**2*

In [166]:
# assume that epsilon and M are small!
expanded_e2.subs({M**2: 0, epsilon**2: 0, M * epsilon: 0})

24.0*M*P*R*UgA_star**2/(128.0*M*R**4/(-8.0*M*R + 16.0*R**2*epsilon + 4.0*R**2) + 512.0*M*R*R**3/(-8.0*M*R + 16.0*R**2*epsilon + 4.0*R**2) + 64.0*M*R**3/(2.0*M - 4.0*R*epsilon - 2.0*R) + 192.0*M*R*R**2/(2.0*M - 4.0*R*epsilon - 2.0*R) - 512.0*R*R**4*epsilon/(-8.0*M*R + 16.0*R**2*epsilon + 4.0*R**2) - 128.0*R*R**4/(-8.0*M*R + 16.0*R**2*epsilon + 4.0*R**2) - 256.0*R*R**3*epsilon/(2.0*M - 4.0*R*epsilon - 2.0*R) - 64.0*R*R**3/(2.0*M - 4.0*R*epsilon - 2.0*R)) - 48.0*M*P*R*UgA_star/(128.0*M*R**4/(-8.0*M*R + 16.0*R**2*epsilon + 4.0*R**2) + 512.0*M*R*R**3/(-8.0*M*R + 16.0*R**2*epsilon + 4.0*R**2) + 64.0*M*R**3/(2.0*M - 4.0*R*epsilon - 2.0*R) + 192.0*M*R*R**2/(2.0*M - 4.0*R*epsilon - 2.0*R) - 512.0*R*R**4*epsilon/(-8.0*M*R + 16.0*R**2*epsilon + 4.0*R**2) - 128.0*R*R**4/(-8.0*M*R + 16.0*R**2*epsilon + 4.0*R**2) - 256.0*R*R**3*epsilon/(2.0*M - 4.0*R*epsilon - 2.0*R) - 64.0*R*R**3/(2.0*M - 4.0*R*epsilon - 2.0*R)) + 24.0*M*P*R/(128.0*M*R**4/(-8.0*M*R + 16.0*R**2*epsilon + 4.0*R**2) + 512.0*M*R*R**3/(

### Basic Model

In [106]:
basic_model_eigenvals = {k.subs({x1: x1_star}).subs({M:0, m:0, epsilon:0}): v for (k, v) in eigenvals.items()}

In [107]:
e1, e2, e3 = basic_model_eigenvals.keys()

### Model with metabolic costs

In [110]:
metabolic_costs_eigenvals = {k.subs({x1: x1_star}).subs({epsilon:0}): v for (k, v) in eigenvals.items()}

### Model with mutation

In [111]:
mutation_eigenvals = {k.subs({x1: x1_star}).subs({M:0, m:0}): v for (k, v) in eigenvals.items()}

## Only the g allele present

Set $x4=1-x3$, $x1=x2=0$ so that only the g allele of the $\gamma$ gene is present.

# Monomorphic $\alpha$ models

## Only A allele present 

Set $x_3=1-x_1$, $x_2=x4=0$ so that only the A allele of the $\alpha$ gene is present

## Only a allele present 

Set $x_4=1-x_2$, $x_1=x3=0$ so that only the a allele of the $\alpha$ gene is present